In [46]:
import matplotlib, cv2
import numpy as np
import matplotlib.pyplot as plt
import math
import os
%matplotlib inline

## Functions to Optimise Edge Detection and Calculate Angles

In [47]:
def optimise_hough(edges):
    optimised = False
    threshold = 50
    lines = cv2.HoughLines(edges,1, np.pi/180, threshold)
    while not optimised:
        if lines.size > 4:
            threshold += 1
            lines = cv2.HoughLines(edges,1, np.pi/180, threshold)
        else:
            lines = cv2.HoughLines(edges,1, np.pi/180, threshold-1)
            optimised = True
    return lines, threshold

In [48]:
def find_thetas(lines, img):
    v_1 = []
    v_2 = []
    gradients = []
    thetas = []
    for line in lines:
        rho,theta = line[0]

        a = np.cos(theta)
        b = np.sin(theta)

        x0 = a * rho
        y0 = b * rho
        # x1 stores the rounded off value of (r * cos(theta) - 1000 * sin(theta))
        x1 = int(x0 + 1000 * (-b))
        # y1 stores the rounded off value of (r * sin(theta)+ 1000 * cos(theta))
        y1 = int(y0 + 1000 * (a))
        # x2 stores the rounded off value of (r * cos(theta)+ 1000 * sin(theta))
        x2 = int(x0 - 1000 * (-b))
        # y2 stores the rounded off value of (r * sin(theta)- 1000 * cos(theta))
        y2 = int(y0 - 1000 * (a))

        if rho < 0:
            theta -= np.pi
        thetas.append(theta)

        cv2.line(img, (x1, y1), (x2, y2), (255, 0, 0), 1)

        v_1.append(np.array([x1,y1]))
        v_2.append(np.array([x2,y2]))

        #gradient = (y2-y1)/(x2-x1)
        #gradients.append(gradient)
    return thetas

In [49]:
def final_angles(thetas):
    results = []
    thetas = list(set(thetas))
    for x in range(len(thetas)):
        for y in range(len(thetas)):
            results.append(abs(math.degrees(thetas[x]-thetas[y])))
    return results


## Final Function to calculate angles of a list of images 

In [50]:
#Input: Image list and File name list
#Output: Dictionary containing image names and calculated angle
def get_image_angles(images, file_names):
    final_res = []
    for image in images:
        edges = cv2.Canny(image, threshold1=70, threshold2=110)
        lines, threshold = optimise_hough(edges)
        thetas = find_thetas(lines, image)
        results = list(set(final_angles(thetas)))

        final_res.append(results[1])
   
    return(dict(zip(file_names, final_res)))
    

## Calculate Angles of .png images in a given directory

In [53]:
path_of_directory = 'angle/'
ext = ('.png')
images = []
file_names = []
for file in os.listdir(path_of_the_directory):
    if file.endswith(ext):
        images.append(cv2.imread(path_of_directory+file))
        file_names.append(file)
        #print(file)
    else:
        continue
        
get_image_angles(images, file_names)

{'image8.png': 108.99999916260987,
 'image9.png': 18.000000500895634,
 'image10.png': 90.00000250447816,
 'image7.png': 94.99999991138924,
 'image6.png': 72.00000200358254,
 'image4.png': 120.00000516053706,
 'image5.png': 53.00000215772734,
 'image1.png': 39.99999883637168,
 'image2.png': 20.000001125733135,
 'image3.png': 60.000001669652114}